In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [2]:
def preprocessing():
    dataset = pd.read_csv("data/car_evaluation.csv")
    
    categorical_columns = ["price", "maint", "doors", "persons", "lug_capacity", "safety"]
    for category in categorical_columns:
        dataset[category] = dataset[category].astype("category")

    price = dataset["price"].cat.codes.values
    maint = dataset["maint"].cat.codes.values
    doors = dataset["doors"].cat.codes.values
    persons = dataset["persons"].cat.codes.values
    lug_capacity = dataset["lug_capacity"].cat.codes.values
    safety = dataset["safety"].cat.codes.values
    
    categorical_data =  np.stack([price, maint, doors, persons, lug_capacity, safety],1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
    
    categorical_colum_sizes = [len(dataset[column].cat.categories) for column in categorical_columns]
    categorical_embedding_sizes = [(col_size, min(50, (col_size)+1) // 2) for col_size in categorical_colum_sizes]

    outputs = dataset["output"].astype("category").cat.codes.values
    outputs = torch.tensor(outputs, dtype=torch.int64).flatten()

    return categorical_data, outputs, categorical_embedding_sizes
    
categorical_data, outputs, categorical_embedding_sizes = preprocessing()

In [3]:
X_train, X_test, y_train, y_test = train_test_split(categorical_data,
                                                    outputs,
                                                    test_size = 0.2,
                                                    random_state=42)

In [4]:
class CarEvalutationDataset(Dataset):
    def __init__(self, categorical_data, outputs):
        self.categorical_data = categorical_data
        self.outputs = outputs

    def __len__(self):
        return len(self.categorical_data)

    def __getitem__(self, idx):
        X = self.categorical_data[idx]
        y = self.outputs[idx]
        return X, y

In [5]:
train_dataset = CarEvalutationDataset(X_train, y_train)
test_dataset = CarEvalutationDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)
        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i
        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical):
        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))
    
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x          

In [7]:
model = Model(categorical_embedding_sizes, 4, [200, 100, 50], p=0.4)
model

Model(
  (all_embeddings): ModuleList(
    (0-2): 3 x Embedding(4, 2)
    (3-5): 3 x Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
epochs = 500
aggregated_losses = []
for epoch in range(epochs):
    model.train()
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch)
        single_loss = loss_function(y_pred, y_batch)
        aggregated_losses.append(single_loss.item())

        optimizer.zero_grad()
        single_loss.backward()
        optimizer.step()

    if (epoch + 1) % 25 == 0:
        print(single_loss.item())

0.6326224207878113
0.33277058601379395
0.538512110710144
0.374930739402771
0.46856096386909485
0.2841736078262329
0.3169310688972473
0.3055759370326996
0.3832331895828247
0.38506484031677246
0.5525647401809692
0.3017103970050812
0.431956946849823
0.4175136387348175
0.27105069160461426
0.5132579207420349
0.14473064243793488
0.42707559466362
0.2564108371734619
0.2940017282962799
